In [1]:
import xlrd
import openpyxl
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import string
import os
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

In [2]:
holiday_path = "휴일_데이터셋/"
path = "data/년월별/"

In [3]:
year = input("enter a year: ")

import os

month_ea = 12
for i in range(1, month_ea+1):
        globals()['month_{}'.format(i)] = pd.read_csv(path+'{}_{}.csv'.format(year,i), encoding='cp949')

enter a year: 2020


In [4]:
globals()['휴일_{}'.format(year)] = pd.read_csv(holiday_path+'휴일_{}.csv'.format(year), encoding='cp949')
globals()['휴일_{}'.format(year)].drop(['holiday','년'], axis=1 ,inplace=True)
globals()['휴일_{}'.format(year)]['휴일'] = int(1)

In [5]:
for i in range(1, month_ea+1):
        globals()['month_{}'.format(i)].drop(['대여대여소명','대여거치대','이용시간','이용거리','반납대여소번호','반납대여소명','반납거치대', '반납년', '반납월', '반납일', '반납시', '반납요일', '반납 지역', '반납대여소 위도', '반납대여소 경도','대여대여소 위도','대여대여소 경도'], axis=1, inplace=True)

In [6]:
for i in range(1, month_ea+1): # make 년월일시
    cols = ['대여년','대여월','대여일','대여시']
    globals()['month_{}'.format(i)]['년월일시'] = globals()['month_{}'.format(i)][cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

In [7]:
for i in range(1, month_ea+1): # merge 휴일, 년월별
    globals()['df_{}'.format(i)] = pd.merge(left=globals()['month_{}'.format(i)],
                                           right=globals()['휴일_{}'.format(year)],
                                           left_on=['대여월', '대여일'],
                                           right_on=['월','일'],
                                           how='left',
                                           sort='False') 

In [8]:
for i in range(1, month_ea+1): # 휴일 판별, 필요없는 거 drop
    globals()['df_{}'.format(i)]['휴일'].fillna(0, inplace=True) #휴일이 아니면 0
    globals()['df_{}'.format(i)].loc[(globals()['df_{}'.format(i)]['휴일'] != 0), '휴일'] = 1
    globals()['df_{}'.format(i)].drop(['대여년','대여월','대여일','대여시','대여요일','월','일'], 
                                      axis=1, inplace=True)

In [9]:
for i in range(1, month_ea+1): # 대여수 추가
    globals()['df_{}'.format(i)]['대여수'] = 1
    globals()['groupby_{}'.format(i)] = globals()['df_{}'.format(i)].set_index(['대여대여소번호','년월일시']).groupby(['대여대여소번호', '년월일시']).sum()

In [10]:
for i in range(1, month_ea+1): # 휴일 원래대로
    globals()['groupby_{}'.format(i)].loc[(globals()['groupby_{}'.format(i)]['휴일'] != 0), '휴일'] = 1
    globals()['groupby_{}'.format(i)].reset_index(level=['대여대여소번호','년월일시'], inplace=True)

In [11]:
for i in range(1, month_ea+1): # csv로 저장
    globals()['groupby_{}'.format(i)].to_csv('년월별+휴일/년월별+휴일_{}_{}.csv'.format(year,i), 
                                             encoding='cp949', index=False)